In [ ]:
!pip install stellargraph

In [ ]:

import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, HinSAGE, link_classification

from tensorflow import keras
from sklearn import preprocessing, feature_extraction, model_selection

from stellargraph import globalvar
from stellargraph import datasets
from IPython.display import display, HTML
%matplotlib inline

In [ ]:
dataset = pd.read_csv('drive/MyDrive/dataset_corrected.csv')

In [ ]:
epochs = 300
embedding_dimension = 100
negative_samples = 2

In [ ]:
# wn18 = datasets.WN18()
# display(HTML(wn18.description))
# wn18_graph, wn18_train, wn18_test, wn18_valid = wn18.load()

In [ ]:
# edges = pd.read_csv('drive/MyDrive/edges.csv')
# values = pd.read_csv( 'drive/MyDrive/values.csv')
# values.set_index('Unnamed: 0', inplace=True)
# # values = values.iloc[1: , :]
# print(values.head())

In [ ]:
source = []
target = []
author_count ={}
count = 0
for index,row in dataset.iterrows():
  # if index == 100000:
  #   break
  if row['Authors'] is not np.nan and row['Citations'] is not np.nan:
    temp = row['Citations'].split(',')
    temp2  = row['Authors'].split(',')
    for i in temp:
      if dataset.iloc[int(i)]['Authors'] is not np.nan:
        temp3 =  dataset.iloc[int(i)]['Authors'].split(',')
        list1_permutations =list(itertools.product(temp2,temp3))
        for perm in list1_permutations:
            if perm[0] in author_count.keys():
              author_count[perm[0]]+=1
            else:
              author_count[perm[0]]=1
            if perm[1] in author_count.keys():
              author_count[perm[1]]+=1
            else:
              author_count[perm[1]]=1
            source.append(perm[0])
            target.append(perm[1])

In [ ]:
values = pd.DataFrame(
    {'Count' : author_count.values()}, index = author_count.keys()
)

from stellargraph import StellarGraph
edges = pd.DataFrame(
    {"source": source, "target": target}
)
Gs = StellarGraph(values, edges)
print(edges)

                        source              target
0                Shinto Eguchi       Noboru Murata
1                Shinto Eguchi  Takashi Takenouchi
2                Shinto Eguchi   Takafumi Kanamori
3                Shinto Eguchi       Shinto Eguchi
4                   John Copas       Noboru Murata
...                        ...                 ...
4080736  Alexandros Potamianos      Qian Ying Wang
4080737      Manolis Perakakis        Jennifer Lai
4080738      Manolis Perakakis  Nicole Yankelovich
4080739  Alexandros Potamianos        Jennifer Lai
4080740  Alexandros Potamianos  Nicole Yankelovich

[4080741 rows x 2 columns]


In [ ]:
print(values.index)

Index(['Shinto Eguchi', 'Noboru Murata', 'Takashi Takenouchi',
       'Takafumi Kanamori', 'John Copas', 'Michael Rosenblum',
       'Maurizio A. Bonuccelli', 'M. Claudia Clò', 'Constantine Caramanis',
       'Michel X. Goemans',
       ...
       'Heinz Roth', 'Alex Simov', 'Vassil Momtchev', 'Mihail Konstantinov',
       'Jérôme Simonin', 'Nicole C. Krämer', 'Nina Simons',
       'Chaitanya Garikiparthi', 'Kenneth Mitchell', 'Ken Mitchell'],
      dtype='object', length=250319)


In [ ]:
from stellargraph import StellarGraph
Gs = StellarGraph(values, edges)

In [ ]:
print(Gs.info())

StellarGraph: Undirected multigraph
 Nodes: 250319, Edges: 4080741

 Node types:
  default: [250319]
    Features: float32 vector, length 1
    Edge types: default-default->default

 Edge types:
    default-default->default: [4080741]
        Weights: all 1 (default)
        Features: none


In [ ]:
from stellargraph.data import EdgeSplitter
# Define an edge splitter on the original graph G:
edge_splitter_test = EdgeSplitter(Gs)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G, and obtain the
# reduced graph G_test with the sampled links removed:
G_test, edge_ids_test, edge_labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global", keep_connected=True
)

** Sampled 408074 positive and 408074 negative edges. **


In [ ]:

# Define an edge splitter on the reduced graph G_test:
edge_splitter_train = EdgeSplitter(G_test)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from G_test, and obtain the
# reduced graph G_train with the sampled links removed:
G_train, edge_ids_train, edge_labels_train = edge_splitter_train.train_test_split(
    p=0.1, method="global", keep_connected=True
)

** Sampled 367266 positive and 367266 negative edges. **


In [ ]:

print(G_train.info())

StellarGraph: Undirected multigraph
 Nodes: 250319, Edges: 3305401

 Node types:
  default: [250319]
    Features: float32 vector, length 1
    Edge types: default-default->default

 Edge types:
    default-default->default: [3305401]
        Weights: all 1 (default)
        Features: none


In [ ]:
print(G_test.info())

StellarGraph: Undirected multigraph
 Nodes: 250319, Edges: 3672667

 Node types:
  default: [250319]
    Features: float32 vector, length 1
    Edge types: default-default->default

 Edge types:
    default-default->default: [3672667]
        Weights: all 1 (default)
        Features: none


In [ ]:
batch_size = 20
epochs = 20

In [ ]:
num_samples = [20, 10]

In [ ]:
train_gen = GraphSAGELinkGenerator(G_train, batch_size, num_samples)
train_flow = train_gen.flow(edge_ids_train, edge_labels_train, shuffle=True)

In [ ]:

test_gen = GraphSAGELinkGenerator(G_test, batch_size, num_samples)
test_flow = test_gen.flow(edge_ids_test, edge_labels_test)

In [ ]:

layer_sizes = [20, 20]
graphsage = GraphSAGE(
    layer_sizes=layer_sizes, generator=train_gen, bias=True, dropout=0.3
)

In [ ]:

# Build the model and expose input and output sockets of graphsage model
# for link prediction
x_inp, x_out = graphsage.in_out_tensors()

In [ ]:
prediction = link_classification(
    output_dim=1, output_act="relu", edge_embedding_method="ip"
)(x_out)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [ ]:
model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=["acc"],
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:

init_train_metrics = model.evaluate(train_flow)
init_test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

40808/40808 [==============================] - 1845s 45ms/step - loss: 1.4489 - acc: 0.4670

Train Set Metrics of the initial (untrained) model:
	loss: 1.4484
	acc: 0.4687

Test Set Metrics of the initial (untrained) model:
	loss: 1.4489
	acc: 0.4670


In [ ]:
history = model.fit(train_flow, epochs=epochs, validation_data=test_flow, verbose=2)

Epoch 1/20


In [ ]:
sg.utils.plot_history(history)

In [ ]:

train_metrics = model.evaluate(train_flow)
test_metrics = model.evaluate(test_flow)

print("\nTrain Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nTest Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))